#Surface Analysis

#Hydration Analysis of 70% of maximum packing density SAMs

###Working Programs

In [4]:
master_set1 = [[],[],[],[],[],[]]

#setup
%matplotlib inline
import numpy as np
import MDAnalysis as md
import matplotlib.pylab as plt
import pandas
#curve fitting setup
from lmfit import Model    
def decay(t, A, tau):
    return A*np.exp(-t/tau)
def myround(x, base):
    return (float(base) * round(float(x)/float(base)))

for runnumber in range(1,7):
    data = pandas.read_csv('%s_2to5ns_50headgroups' % runnumber, header=None, names=['time', \
    'dx', 'dy', 'dz'])
    tcom = data[['time']].values
    xcom = data['dx'].values
    ycom = data['dy'].values
    zcom = data['dz'].values
    start = np.mean(zcom)
    start = myround(start, 0.25) #to match density plots
    start = start*10 #convert to angstroms
    
    #begin analysis centered around COM of headgroups (2 angstrom slices)
    l1 = start -1

    PDB = "50.pdb"
    XTC = "set1_%s_50.xtc" % runnumber # Enter filename for trajectory
    u = md.Universe(PDB, XTC)
    SOL = u.selectAtoms("resname SOL")
    tauvalues = []

    for layer in range(0,40): #Increment up to 10 A from surface
        tauvalues.append(layer) #layer=0 in first iteration, we then increment by 0.25 
        currentlayer = l1 + 0.25 * layer #angstroms.layers are in 0.25 angstrom slices 
        nextlayer = currentlayer + 2 #(we're moving a 2 A window in 0.25 A steps in Z)
        currentlayer = str('%.4g' % currentlayer)
        nextlayer = str('%.4g' % nextlayer)
        print "prop z >= %s and prop z <= %s" % (currentlayer, nextlayer)

        timeslice = ['Ct', 'Dt', 'Et', 'Ft', 'Gt', 'Ht', 'It', 'Jt', 'Kt', 'Lt', 'Mt']
        n = ['Cn', 'Dn', 'En', 'Fn', 'Gn', 'Hn', 'In', 'Jn', 'Kn', 'Ln']
        Cty = 0

        for i in range(10):
            j = i * 100 +2000 #shift analysis to start at 2 ns. Time slices are in 100 ps 
            k = j + 40 #increments, and we read from 0-100ps to 900-1000ps
            for ts in u.trajectory[j:(j+1)]: #read the trajectory for 1 ps
                slice0 = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
            n[i] = len(slice0) #n stores the total number of initial waters found
            init_res = []
            for x in slice0:
                init_res.append(x) #append the initial resids to init_res
            
                     
            timeslice[i] = []
            for t in u.trajectory[j:k]: #read the trajectory incrementally in the current time 
                p = 0.25                #window (100 ps)
                reslist = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
                for x in init_res:
                    if x in reslist:
                        p+=1
                    else:
                        init_res.remove(x) #if the resid is not found, remove it from init_res
                timeslice[i].append((u.trajectory.time, p))
            timeslice[i] = np.array(timeslice[i]) #make timeslice[] an array containing the time 
                                                  #stamp (100ps window) and number of waters 
            Cty = (timeslice[i][:,1]/n[i]) + Cty  #present (p)
                                                  #Cty is incorrect after the initial value 
                                                  #of i until it is divided by 10 later in the 
                                                  #code. It is every ps window divided by the 
        Cty = Cty/10                              #original # of waters for that window (n[]). 
        #fig = plt.figure(figsize=(12,8))         #Essentially  it is timslice[] with fraction 
                                                  #remaining waters instead of #of waters. 
        #axes = fig.add_subplot(111)              #     def decay(t, A, tau):
        #axes.plot(timeslice[0][:,0], Cty, 'r-')  #     return A*np.exp(-t/tau)
        #plt.ylabel('Autocorrelation')            #we want to solve for A and tau that gives the  
        #plt.xlabel('Time (ps)')         #the exponential decay of waters in the timeslices
        #plt.grid(True)                  #http://mx.nthu.edu.tw/~cchu/course/acrk/chapter7.pdf
        #plt.xlim(0,100)                 #res time fitting to an exponential decay is common
        #plt.show()                      #our model here mimics that of a CSTR E(t)=A*e^(-t/tau)

        #print Cty
        
        model = Model(decay, independent_vars=['t'])
        result = model.fit(Cty, t=np.array(range(1,41)), A=1, tau=80)
        print result.values
        temp = result.values
        tauvalues[layer] = temp['tau']
        temp = 0
    
    accessmaster = runnumber - 1
    master_set1[accessmaster] = tauvalues

prop z >= 14 and prop z <= 16
{'A': 1.1756641887685839, 'tau': 3.4405645862452348}
prop z >= 14.25 and prop z <= 16.25
{'A': 1.2071413060004701, 'tau': 3.456339304147328}
prop z >= 14.5 and prop z <= 16.5
{'A': 1.2163744833696268, 'tau': 3.3675667213917917}
prop z >= 14.75 and prop z <= 16.75
{'A': 1.2243101325739361, 'tau': 3.300559303781879}
prop z >= 15 and prop z <= 17
{'A': 1.2227448779293257, 'tau': 3.3342496449659516}
prop z >= 15.25 and prop z <= 17.25
{'A': 1.2286246274802306, 'tau': 3.2707674734869769}
prop z >= 15.5 and prop z <= 17.5
{'A': 1.216592612911366, 'tau': 3.2934946585136999}
prop z >= 15.75 and prop z <= 17.75
{'A': 1.218101702091438, 'tau': 3.3510727431663287}
prop z >= 16 and prop z <= 18
{'A': 1.2269194086547865, 'tau': 3.3372963557656368}
prop z >= 16.25 and prop z <= 18.25
{'A': 1.216159057080868, 'tau': 3.4087296541941989}
prop z >= 16.5 and prop z <= 18.5
{'A': 1.2118932047651998, 'tau': 3.4386309013664795}
prop z >= 16.75 and prop z <= 18.75
{'A': 1.210896

In [ ]:
master_set2 = [[],[],[],[],[],[]]

#setup
%matplotlib inline
import numpy as np
import MDAnalysis as md
import matplotlib.pylab as plt
import pandas
#curve fitting setup
from lmfit import Model    
def decay(t, A, tau):
    return A*np.exp(-t/tau)
def myround(x, base):
    return (float(base) * round(float(x)/float(base)))

for runnumber in range(1,7):
    data = pandas.read_csv('%s_2to5ns_50headgroups' % runnumber, header=None, names=['time', \
    'dx', 'dy', 'dz'])
    tcom = data[['time']].values
    xcom = data['dx'].values
    ycom = data['dy'].values
    zcom = data['dz'].values
    start = np.mean(zcom)
    start = myround(start, 0.25) #to match density plots
    start = start*10 #convert to angstroms
    
    #begin analysis centered around COM of headgroups (2 angstrom slices)
    l1 = start -1

    PDB = "50.pdb"
    XTC = "set2_%s_50.xtc" % runnumber # Enter filename for trajectory
    u = md.Universe(PDB, XTC)
    SOL = u.selectAtoms("resname SOL")
    tauvalues = []

    for layer in range(0,40): #Increment up to 10 A from surface
        tauvalues.append(layer) #layer=0 in first iteration, we then increment by 0.25 
        currentlayer = l1 + 0.25 * layer #angstroms.layers are in 0.25 angstrom slices 
        nextlayer = currentlayer + 2 #(we're moving a 2 A window in 0.25 A steps in Z)
        currentlayer = str('%.4g' % currentlayer)
        nextlayer = str('%.4g' % nextlayer)
        print "prop z >= %s and prop z <= %s" % (currentlayer, nextlayer)

        timeslice = ['Ct', 'Dt', 'Et', 'Ft', 'Gt', 'Ht', 'It', 'Jt', 'Kt', 'Lt', 'Mt']
        n = ['Cn', 'Dn', 'En', 'Fn', 'Gn', 'Hn', 'In', 'Jn', 'Kn', 'Ln']
        Cty = 0

        for i in range(10):
            j = i * 100 +2000 #shift analysis to start at 2 ns. Time slices are in 100 ps 
            k = j + 40 #increments, and we read from 0-100ps to 900-1000ps
            for ts in u.trajectory[j:(j+1)]: #read the trajectory for 1 ps
                slice0 = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
            n[i] = len(slice0) #n stores the total number of initial waters found
            init_res = []
            for x in slice0:
                init_res.append(x) #append the initial resids to init_res
            
                     
            timeslice[i] = []
            for t in u.trajectory[j:k]: #read the trajectory incrementally in the current time 
                p = 0.25                #window (100 ps)
                reslist = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
                for x in init_res:
                    if x in reslist:
                        p+=1
                    else:
                        init_res.remove(x) #if the resid is not found, remove it from init_res
                timeslice[i].append((u.trajectory.time, p))
            timeslice[i] = np.array(timeslice[i]) #make timeslice[] an array containing the time 
                                                  #stamp (100ps window) and number of waters 
            Cty = (timeslice[i][:,1]/n[i]) + Cty  #present (p)
                                                  #Cty is incorrect after the initial value 
                                                  #of i until it is divided by 10 later in the 
                                                  #code. It is every ps window divided by the 
        Cty = Cty/10                              #original # of waters for that window (n[]). 
        #fig = plt.figure(figsize=(12,8))         #Essentially  it is timslice[] with fraction 
                                                  #remaining waters instead of #of waters. 
        #axes = fig.add_subplot(111)              #     def decay(t, A, tau):
        #axes.plot(timeslice[0][:,0], Cty, 'r-')  #     return A*np.exp(-t/tau)
        #plt.ylabel('Autocorrelation')            #we want to solve for A and tau that gives the  
        #plt.xlabel('Time (ps)')         #the exponential decay of waters in the timeslices
        #plt.grid(True)                  #http://mx.nthu.edu.tw/~cchu/course/acrk/chapter7.pdf
        #plt.xlim(0,100)                 #res time fitting to an exponential decay is common
        #plt.show()                      #our model here mimics that of a CSTR E(t)=A*e^(-t/tau)

        #print Cty
        
        model = Model(decay, independent_vars=['t'])
        result = model.fit(Cty, t=np.array(range(1,41)), A=1, tau=80)
        print result.values
        temp = result.values
        tauvalues[layer] = temp['tau']
        temp = 0
    
    accessmaster = runnumber - 1
    master_set2[accessmaster] = tauvalues

In [ ]:
master_set3 = [[],[],[],[],[],[]]

#setup
%matplotlib inline
import numpy as np
import MDAnalysis as md
import matplotlib.pylab as plt
import pandas
#curve fitting setup
from lmfit import Model    
def decay(t, A, tau):
    return A*np.exp(-t/tau)
def myround(x, base):
    return (float(base) * round(float(x)/float(base)))

for runnumber in range(1,7):
    data = pandas.read_csv('%s_2to5ns_50headgroups' % runnumber, header=None, names=['time', \
    'dx', 'dy', 'dz'])
    tcom = data[['time']].values
    xcom = data['dx'].values
    ycom = data['dy'].values
    zcom = data['dz'].values
    start = np.mean(zcom)
    start = myround(start, 0.25) #to match density plots
    start = start*10 #convert to angstroms
    
    #begin analysis centered around COM of headgroups (2 angstrom slices)
    l1 = start -1

    PDB = "50.pdb"
    XTC = "set3_%s_50.xtc" % runnumber # Enter filename for trajectory
    u = md.Universe(PDB, XTC)
    SOL = u.selectAtoms("resname SOL")
    tauvalues = []

    for layer in range(0,40): #Increment up to 10 A from surface
        tauvalues.append(layer) #layer=0 in first iteration, we then increment by 0.25 
        currentlayer = l1 + 0.25 * layer #angstroms.layers are in 0.25 angstrom slices 
        nextlayer = currentlayer + 2 #(we're moving a 2 A window in 0.25 A steps in Z)
        currentlayer = str('%.4g' % currentlayer)
        nextlayer = str('%.4g' % nextlayer)
        print "prop z >= %s and prop z <= %s" % (currentlayer, nextlayer)

        timeslice = ['Ct', 'Dt', 'Et', 'Ft', 'Gt', 'Ht', 'It', 'Jt', 'Kt', 'Lt', 'Mt']
        n = ['Cn', 'Dn', 'En', 'Fn', 'Gn', 'Hn', 'In', 'Jn', 'Kn', 'Ln']
        Cty = 0

        for i in range(10):
            j = i * 100 +2000 #shift analysis to start at 2 ns. Time slices are in 100 ps 
            k = j + 40 #increments, and we read from 0-100ps to 900-1000ps
            for ts in u.trajectory[j:(j+1)]: #read the trajectory for 1 ps
                slice0 = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
            n[i] = len(slice0) #n stores the total number of initial waters found
            init_res = []
            for x in slice0:
                init_res.append(x) #append the initial resids to init_res
            
                     
            timeslice[i] = []
            for t in u.trajectory[j:k]: #read the trajectory incrementally in the current time 
                p = 0.25                #window (100 ps)
                reslist = SOL.selectAtoms("prop z >= %s and prop z <= %s" % (currentlayer, \
                nextlayer)).residues.resids()
                for x in init_res:
                    if x in reslist:
                        p+=1
                    else:
                        init_res.remove(x) #if the resid is not found, remove it from init_res
                timeslice[i].append((u.trajectory.time, p))
            timeslice[i] = np.array(timeslice[i]) #make timeslice[] an array containing the time 
                                                  #stamp (100ps window) and number of waters 
            Cty = (timeslice[i][:,1]/n[i]) + Cty  #present (p)
                                                  #Cty is incorrect after the initial value 
                                                  #of i until it is divided by 10 later in the 
                                                  #code. It is every ps window divided by the 
        Cty = Cty/10                              #original # of waters for that window (n[]). 
        #fig = plt.figure(figsize=(12,8))         #Essentially  it is timslice[] with fraction 
                                                  #remaining waters instead of #of waters. 
        #axes = fig.add_subplot(111)              #     def decay(t, A, tau):
        #axes.plot(timeslice[0][:,0], Cty, 'r-')  #     return A*np.exp(-t/tau)
        #plt.ylabel('Autocorrelation')            #we want to solve for A and tau that gives the  
        #plt.xlabel('Time (ps)')         #the exponential decay of waters in the timeslices
        #plt.grid(True)                  #http://mx.nthu.edu.tw/~cchu/course/acrk/chapter7.pdf
        #plt.xlim(0,100)                 #res time fitting to an exponential decay is common
        #plt.show()                      #our model here mimics that of a CSTR E(t)=A*e^(-t/tau)

        #print Cty
        
        model = Model(decay, independent_vars=['t'])
        result = model.fit(Cty, t=np.array(range(1,41)), A=1, tau=80)
        print result.values
        temp = result.values
        tauvalues[layer] = temp['tau']
        temp = 0
    
    accessmaster = runnumber - 1
    master_set3[accessmaster] = tauvalues